# Machine Learning Project - Kickstarter Data Set
## Data Cleaning Notebook
*Contributor: Max Langer, René Ebrecht, Jens Reich*

This is the very first project where we build a machine learning model from scratch based on an unknown dataset.
The dataset includes data from Kickstarter projects from the years 2009 to 2019.

Our goal is to help our (fictional) stackholder, PPC Consultants with a model that can predict whether a Kickstarter project will be successful or not. 
PPC Consultants advises potential project creators (PPCs) with their projects to get them off the ground as successfully as possible.
Therefore, the value of our data product (the predictive model) is to show opportunities, save time, and in the end make money for both PPC consultants and PPCs.

In [1]:
# Import the organization modules
import pandas as pd
import numpy as np
# Import module to ignore warnings
import warnings
warnings.filterwarnings('ignore')
# Import the plot modules
import matplotlib.pyplot as plt
import seaborn as sns
# Import own scripts
from scripts.data_cleaning import (
    read_all_csvs, 
    data_cleaning,
    create_csv, 
    get_nan_cols, 
    convert_to_datetime, 
    calculate_time_periods, 
    get_year_month_day, 
    entangle_column
    )

In [2]:
# Create data frame from all single CSV files
df = read_all_csvs()

In [3]:
# Take a look at the first columns
df.head()

,backers_count,blurb,category,converted_pledged_amount,country,created_at,creator,currency,currency_symbol,currency_trailing_code,...,slug,source_url,spotlight,staff_pick,state,state_changed_at,static_usd_rate,urls,usd_pledged,usd_type
0,21,2006 was almost 7 years ago.... Can you believ...,"{""id"":43,""name"":""Rock"",""slug"":""music/rock"",""po...",802,US,1387659690,"{""id"":1495925645,""name"":""Daniel"",""is_registere...",USD,$,True,...,new-final-round-album,https://www.kickstarter.com/discover/categorie...,True,False,successful,1391899046,1.000000,"{""web"":{""project"":""https://www.kickstarter.com...",802.00000,international
1,97,An adorable fantasy enamel pin series of princ...,"{""id"":54,""name"":""Mixed Media"",""slug"":""art/mixe...",2259,US,1549659768,"{""id"":1175589980,""name"":""Katherine"",""slug"":""fr...",USD,$,True,...,princess-pals-enamel-pin-series,https://www.kickstarter.com/discover/categorie...,True,False,successful,1551801611,1.000000,"{""web"":{""project"":""https://www.kickstarter.com...",2259.00000,international
2,88,Helping a community come together to set the s...,"{""id"":280,""name"":""Photobooks"",""slug"":""photogra...",29638,US,1477242384,"{""id"":1196856269,""name"":""MelissaThomas"",""is_re...",USD,$,True,...,their-life-through-their-lens-the-amish-and-me...,https://www.kickstarter.com/discover/categorie...,True,True,successful,1480607932,1.000000,"{""web"":{""project"":""https://www.kickstarter.com...",29638.00000,international
3,193,Every revolution starts from the bottom and we...,"{""id"":266,""name"":""Footwear"",""slug"":""fashion/fo...",49158,IT,1540369920,"{""id"":1569700626,""name"":""WAO"",""slug"":""wearewao...",EUR,€,False,...,wao-the-eco-effect-shoes,https://www.kickstarter.com/discover/categorie...,True,False,successful,1544309940,1.136525,"{""web"":{""project"":""https://www.kickstarter.com...",49075.15252,international
4,20,Learn to build 10+ Applications in this comple...,"{""id"":51,""name"":""Software"",""slug"":""technology/...",549,US,1425706517,"{""id"":1870845385,""name"":""Kalpit Jain"",""is_regi...",USD,$,True,...,apple-watch-development-course,https://www.kickstarter.com/discover/categorie...,False,False,failed,1428511019,1.000000,"{""web"":{""project"":""https://www.kickstarter.com...",549.00000,domestic


In [4]:
# Prints out the columns with NaNs
get_nan_cols(df)

Number of NaNs per column:
blurb:8
friends:208922
is_backing:208922
is_starred:208922
location:226
permissions:208922
usd_type:480


We can get rid of some columns, which probably will have no impact on the model later on. 
Especially `friends`, `is_backing`, `is_starred`, `permission` are almost completely empty (over 50% of data was missing). So no information loss here.

Moreover the following columns don't seem to contain any valuable information for the predictions as well: `blurb`,`converted_pledged_amount`,`currency_symbol`, `currency_trailing_code`, `fx_rate`, `photo`,`profile`,`slug`,`source_url`,`static_usd_rate`,`urls`.

The 226 and 480 missing observations for `location` and `usd_type` are so small in number that we can get rid of these rows as well. 

We also rename the `currency` into `original_currency`, and `id` int `project_id`, as this gives more information about its content. 

In [5]:
# Convert the column names to a list
columns = df.columns.to_list()
# Drop all columns with more than 50% of the observations missing
df = df[[column for column in df if df[column].count() / len(df) >= 0.5]]
# Print the dropped columns
print("Dropped columnes:", end= " ")
dropped = [print(col, end=" ") for col in columns if col not in df.columns]

Dropped columnes: friends is_backing is_starred permissions 

In [6]:
# Drop the listed columns
df.drop([
    'blurb', 
    'converted_pledged_amount',
    'currency_symbol', 
    'currency_trailing_code', 
    'fx_rate',
    'photo',
    'profile',
    'slug',
    'source_url', 
    'static_usd_rate',
    'urls'
    ], axis=1, inplace=True)
# Drops the last few NaN values
df.dropna(axis=0, inplace=True)
# Rename the currency column
df.rename(columns={
    'currency':'original_currency',
    'id':'project_id'}, inplace=True)

In [7]:
# Prints out the columns with NaNs
get_nan_cols(df)

Number of NaNs per column:


Now we have no NaNs left in our data set, which is another step towards a cleaned data set.

The columns of `created_at`, `state_changed_at`, `deadline` are actually dates but expressed as epoch time. We will convert these into datetime objects and thereby into actual dates.

With these we can calculate the time periods from the beginning of the project until the success and also to the deadline of the project.

Moreover, from these dates we can then get the year, month and day of the three columns.

In [8]:
# Convert the time columns to datetime types
df = convert_to_datetime(df, ['created_at', 'state_changed_at', 'deadline'])
# Calculate the time periods
df = calculate_time_periods(df)
# Get the years, months and days as separate columns
df = get_year_month_day(df, ['created_at', 'state_changed_at', 'deadline'])

In [9]:
df.head()

,backers_count,category,country,creator,original_currency,current_currency,disable_communication,goal,project_id,is_starrable,...,days_total,created_at_year,created_at_month,created_at_day,state_changed_at_year,state_changed_at_month,state_changed_at_day,deadline_year,deadline_month,deadline_day
0,21,"{""id"":43,""name"":""Rock"",""slug"":""music/rock"",""po...",US,"{""id"":1495925645,""name"":""Daniel"",""is_registere...",USD,USD,False,200.0,287514992,False,...,49,2013,12,21,2014,2,8,2014,2,8
1,97,"{""id"":54,""name"":""Mixed Media"",""slug"":""art/mixe...",US,"{""id"":1175589980,""name"":""Katherine"",""slug"":""fr...",USD,USD,False,400.0,385129759,False,...,25,2019,2,8,2019,3,5,2019,3,5
2,88,"{""id"":280,""name"":""Photobooks"",""slug"":""photogra...",US,"{""id"":1196856269,""name"":""MelissaThomas"",""is_re...",USD,USD,False,27224.0,681033598,False,...,39,2016,10,23,2016,12,1,2016,12,1
3,193,"{""id"":266,""name"":""Footwear"",""slug"":""fashion/fo...",IT,"{""id"":1569700626,""name"":""WAO"",""slug"":""wearewao...",EUR,USD,False,40000.0,1031782682,False,...,45,2018,10,24,2018,12,8,2018,12,8
4,20,"{""id"":51,""name"":""Software"",""slug"":""technology/...",US,"{""id"":1870845385,""name"":""Kalpit Jain"",""is_regi...",USD,USD,False,1000.0,904085819,False,...,32,2015,3,7,2015,4,8,2015,4,8


In [10]:
print("Percentage of the data where the project duration is the same as the time periode to reach the goal:")
print((df[df['days_till_change']==df['days_total']].shape[0] / len(df))*100)

Percentage of the data where the project duration is the same as the time periode to reach the goal:
92.39243031709798


We see that 92% of the projects reach the goal in the same number of days as the deadline. Perhaps they were simply marked as completed after the target was reached. 

Let's look further into the data types of the other columns.

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 208516 entries, 0 to 964
Data columns (total 30 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   backers_count           208516 non-null  int64  
 1   category                208516 non-null  object 
 2   country                 208516 non-null  object 
 3   creator                 208516 non-null  object 
 4   original_currency       208516 non-null  object 
 5   current_currency        208516 non-null  object 
 6   disable_communication   208516 non-null  bool   
 7   goal                    208516 non-null  float64
 8   project_id              208516 non-null  int64  
 9   is_starrable            208516 non-null  bool   
 10  launched_at             208516 non-null  int64  
 11  location                208516 non-null  object 
 12  name                    208516 non-null  object 
 13  pledged                 208516 non-null  float64
 14  spotlight              

In [12]:
for col in ['category', 'creator', 'location']:
    print(df[col].iloc[0], '\n')

{"id":43,"name":"Rock","slug":"music/rock","position":17,"parent_id":14,"color":10878931,"urls":{"web":{"discover":"http://www.kickstarter.com/discover/categories/music/rock"}}} 

{"id":1495925645,"name":"Daniel","is_registered":null,"chosen_currency":null,"avatar":{"thumb":"https://ksr-ugc.imgix.net/assets/006/041/047/c44d1a95c2139ae46af635c7c6e7ea76_original.jpg?ixlib=rb-1.1.0&w=40&h=40&fit=crop&v=1461362658&auto=format&frame=1&q=92&s=3d655afafac9dbb59c1e675adfa87082","small":"https://ksr-ugc.imgix.net/assets/006/041/047/c44d1a95c2139ae46af635c7c6e7ea76_original.jpg?ixlib=rb-1.1.0&w=160&h=160&fit=crop&v=1461362658&auto=format&frame=1&q=92&s=3973d24f5c3db1ed1d5c84cec8af1d6d","medium":"https://ksr-ugc.imgix.net/assets/006/041/047/c44d1a95c2139ae46af635c7c6e7ea76_original.jpg?ixlib=rb-1.1.0&w=160&h=160&fit=crop&v=1461362658&auto=format&frame=1&q=92&s=3973d24f5c3db1ed1d5c84cec8af1d6d"},"urls":{"web":{"user":"https://www.kickstarter.com/profile/1495925645"},"api":{"user":"https://api.kick

We find very strange formatted object type data in `category`, `creator` and `location` column. The next step will be to entangle these strings and get the most important information out of them. For the location we could extract the city name of the project creator. For the creator we could get their names for now. And for the category the id and the name could be interesting.

In [13]:
# Entangles the category, creator and location column
df = entangle_column(df, ['category', 'creator', 'location'])

In [14]:
df.head()

,backers_count,country,original_currency,current_currency,disable_communication,goal,project_id,is_starrable,launched_at,name,...,state_changed_at_month,state_changed_at_day,deadline_year,deadline_month,deadline_day,category_name,category_id,category_sub,creator_name,location_name
0,21,US,USD,USD,False,200.0,287514992,False,1388011046,New Final Round Album,...,2,8,2014,2,8,music,43,rock,Daniel,Chicago
1,97,US,USD,USD,False,400.0,385129759,False,1550073611,Princess Pals Enamel Pin Series,...,3,5,2019,3,5,art,54,mixed media,Katherine,Sacramento
2,88,US,USD,USD,False,27224.0,681033598,False,1478012330,Their Life Through Their Lens-the Amish and Me...,...,12,1,2016,12,1,photography,280,photobooks,MelissaThomas,Columbus
3,193,IT,EUR,USD,False,40000.0,1031782682,False,1540684582,WAO: THE ECO EFFECT SHOES,...,12,8,2018,12,8,fashion,266,footwear,WAO,Venice
4,20,US,USD,USD,False,1000.0,904085819,False,1425919017,Apple Watch Development Course,...,4,8,2015,4,8,technology,51,software,Kalpit Jain,Redmond


In [15]:
get_nan_cols(df)

Number of NaNs per column:


Now we should take a look at the number of unique observations for each column.

In [16]:
# Show the amount of unique items per column
df.nunique()

backers_count               3237
country                       22
original_currency             14
current_currency               1
disable_communication          2
goal                        5103
project_id                182004
is_starrable                   2
launched_at               181849
name                      181421
pledged                    44293
spotlight                      2
staff_pick                     2
state                          5
usd_pledged                79114
usd_type                       2
days_till_change            1501
days_total                  1497
created_at_year               11
created_at_month              12
created_at_day                31
state_changed_at_year         11
state_changed_at_month        12
state_changed_at_day          31
deadline_year                 11
deadline_month                12
deadline_day                  31
category_name                 15
category_id                  169
category_sub                 145
creator_na

If we look at the number of unique entries for each column, we can see that the current currency has only one entry, USD. Since this does not add any information to the model, we can also drop this column. 

In [17]:
# Drop the current_currency column
df.drop('current_currency', axis=1, inplace=True)

In [18]:
df.head()

,backers_count,country,original_currency,disable_communication,goal,project_id,is_starrable,launched_at,name,pledged,...,state_changed_at_month,state_changed_at_day,deadline_year,deadline_month,deadline_day,category_name,category_id,category_sub,creator_name,location_name
0,21,US,USD,False,200.0,287514992,False,1388011046,New Final Round Album,802.0,...,2,8,2014,2,8,music,43,rock,Daniel,Chicago
1,97,US,USD,False,400.0,385129759,False,1550073611,Princess Pals Enamel Pin Series,2259.0,...,3,5,2019,3,5,art,54,mixed media,Katherine,Sacramento
2,88,US,USD,False,27224.0,681033598,False,1478012330,Their Life Through Their Lens-the Amish and Me...,29638.0,...,12,1,2016,12,1,photography,280,photobooks,MelissaThomas,Columbus
3,193,IT,EUR,False,40000.0,1031782682,False,1540684582,WAO: THE ECO EFFECT SHOES,43180.0,...,12,8,2018,12,8,fashion,266,footwear,WAO,Venice
4,20,US,USD,False,1000.0,904085819,False,1425919017,Apple Watch Development Course,549.0,...,4,8,2015,4,8,technology,51,software,Kalpit Jain,Redmond


In [19]:
# Create a list of the real numeric columns
numeric_cols = [
    'backers_count', 
    'goal', 
    'pledged',
    'usd_pledged',
    'days_till_change',
    'days_total']
# Get the basic descriptive statistics for the numeric columns
df[numeric_cols].describe()

,backers_count,goal,pledged,usd_pledged,days_till_change,days_total
count,208516.000000,2.085160e+05,2.085160e+05,2.085160e+05,208516.000000,208516.000000
mean,145.277734,4.926378e+04,1.881146e+04,1.289545e+04,78.145221,80.340765
std,884.468471,1.181398e+06,3.234143e+05,8.896665e+04,132.063629,131.917357
min,0.000000,1.000000e-02,0.000000e+00,0.000000e+00,0.000000,1.000000
25%,4.000000,1.500000e+03,1.090000e+02,1.060000e+02,32.000000,33.000000
50%,27.000000,5.000000e+03,1.555000e+03,1.537000e+03,45.000000,47.000000
75%,89.000000,1.500000e+04,6.889057e+03,6.552000e+03,71.000000,73.000000
max,105857.000000,1.000000e+08,8.103074e+07,8.596475e+06,3334.000000,3334.000000


In [21]:
df.groupby(['category_id', 'category_name']).count()

,,backers_count,country,original_currency,disable_communication,goal,project_id,is_starrable,launched_at,name,pledged,...,created_at_day,state_changed_at_year,state_changed_at_month,state_changed_at_day,deadline_year,deadline_month,deadline_day,category_sub,creator_name,location_name
category_id,category_name,,,,,,,,,,,,,,,,,,,,,
1,art,1019,1019,1019,1019,1019,1019,1019,1019,1019,1019,...,1019,1019,1019,1019,1019,1019,1019,1019,1019,1019
10,food,568,568,568,568,568,568,568,568,568,568,...,568,568,568,568,568,568,568,568,568,568
11,film & video,276,276,276,276,276,276,276,276,276,276,...,276,276,276,276,276,276,276,276,276,276
12,games,95,95,95,95,95,95,95,95,95,95,...,95,95,95,95,95,95,95,95,95,95
13,journalism,699,699,699,699,699,699,699,699,699,699,...,699,699,699,699,699,699,699,699,699,699
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53,art,2496,2496,2496,2496,2496,2496,2496,2496,2496,2496,...,2496,2496,2496,2496,2496,2496,2496,2496,2496,2496
54,art,2546,2546,2546,2546,2546,2546,2546,2546,2546,2546,...,2546,2546,2546,2546,2546,2546,2546,2546,2546,2546
6,dance,1478,1478,1478,1478,1478,1478,1478,1478,1478,1478,...,1478,1478,1478,1478,1478,1478,1478,1478,1478,1478


In [4]:
df.head()

,backers_count,country,original_currency,disable_communication,goal,project_id,is_starrable,launched_at,name,pledged,...,state_changed_at_month,state_changed_at_day,deadline_year,deadline_month,deadline_day,category_name,category_id,category_sub,creator_name,location_name
0,21,US,USD,False,200.0,287514992,False,1388011046,New Final Round Album,802.0,...,2,8,2014,2,8,music,43,rock,Daniel,Chicago
1,97,US,USD,False,400.0,385129759,False,1550073611,Princess Pals Enamel Pin Series,2259.0,...,3,5,2019,3,5,art,54,mixed media,Katherine,Sacramento
2,88,US,USD,False,27224.0,681033598,False,1478012330,Their Life Through Their Lens-the Amish and Me...,29638.0,...,12,1,2016,12,1,photography,280,photobooks,MelissaThomas,Columbus
3,193,IT,EUR,False,40000.0,1031782682,False,1540684582,WAO: THE ECO EFFECT SHOES,43180.0,...,12,8,2018,12,8,fashion,266,footwear,WAO,Venice
4,20,US,USD,False,1000.0,904085819,False,1425919017,Apple Watch Development Course,549.0,...,4,8,2015,4,8,technology,51,software,Kalpit Jain,Redmond
